# Tutorial 2 - AquaHarmonics
The goal of this tutorial is to illustrate a more realistic model of a PTO, including non-linear power conversion chain. 
It uses the [AquaHarmonics](https://aquaharmonics.com/wec_vis/) device in one degree of freedom in regular waves and models the PTO generator using a non-linear efficiency map and adds realistic constraints, including generator maximum torque and min/max line tension.

![AquaHarmonics device](https://aquaharmonics.com/wec_vis.png)

In [ ]:
import capytaine as cpy
import autograd.numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.optimize import brute

import wecopttool as wot

## 1. Optimal control with non-liner power conversion chain

### WEC object
We will go through a number of steps to create a `WEC` object, with which we can solve for the optimal control trajectory.
To compose the `WEC` object, we will need to...

TODO

#### Geometry
First we create a surface mesh for the WEC hull.
We can quickly visualize the cross-sectional shape of the hull.

In [ ]:
ah_hull = wot.geom.AquaHarmonics()
mesh = ah_hull.mesh(mesh_size_factor=0.25)
_ = ah_hull.plot_cross_section()

Next we create a `FloatingBody` object from the mesh in Capytaine, which we will use to calculate the hydrodynamics.
We can visualize the a 3D rendering of the mesh now as well.

In [ ]:
fb = cpy.FloatingBody.from_meshio(mesh, name="AquaHarmonics")
fb.add_translation_dof(name="Heave")
ndof = fb.nb_dofs
fb.show_matplotlib()

#### Hydrostatics and mass
The AquaHarmonics device is positively buoyant (i.e., the buoyancy force and greater than the force due to gravity).
Therefore, we'll calculate the hydrostatic effects from the mesh, but set rigid-body mass manually.

In [ ]:
mass = np.atleast_2d(5500) # [kg] mass w. ballast
stiffness = wot.hydrostatics.stiffness_matrix(fb).values

# will be needed for additional forces/constraints.
g = 9.81
rho = 1025
displaced_mass_cpy = wot.hydrostatics.inertia_matrix(fb, rho).values
displacement = displaced_mass_cpy/rho # [m^3] disp. vol w. ballast and pretension

#### Hydrodynamics

Next we will run the boundary element model (BEM; [Capytaine](https://github.com/capytaine/capytaine)) to obtaine the hydrodynamics for the hull.
We can also quickly plot the results to confirm they look reasonable.
Before running the BEM solver, we must specify a set of frequencies at which to perform the calculations.
For `WecOptTool`, these must be a regularly spaced frequency array.

In [ ]:
f1 = 0.08 # [Hz]
nfreq = 10
freq = wot.frequency(f1, nfreq, False) # False -> no zero frequency

bem_data = wot.run_bem(fb, freq, rho=rho, g=g)

bem_file = "./bem_data.nc"
wot.write_netcdf(bem_file, bem_data)

In [ ]:
fig, axes = plt.subplots(3,1)
bem_data['added_mass'].plot(ax = axes[0])
bem_data['radiation_damping'].plot(ax = axes[1])
axes[2].plot(bem_data['omega'],np.abs(np.squeeze(bem_data['diffraction_force'].values)), color = 'orange')
axes[2].set_ylabel('abs(diffraction_force)', color = 'orange')
axes[2].tick_params(axis ='y', labelcolor = 'orange')
ax2r = axes[2].twinx()
ax2r.plot(bem_data['omega'],np.abs(np.squeeze(bem_data['Froude_Krylov_force'].values)), color = 'blue')
ax2r.set_ylabel('abs(FK_force)', color = 'blue')
ax2r.tick_params(axis ='y', labelcolor = 'blue')

for axi in axes:
    axi.set_title('')
    axi.label_outer()
    axi.grid()
    
axes[-1].set_xlabel('Frequency [rad/s]')

#### PTO 
The AquaHarmonics device drive train includes a generator and pneumatic air spring, each of which have distinct gearings relative to linear tether motion as well as finite levels of inertia and friction.
Here, we define each of these factors.

![PTO diagram](./PTO.png)

In [ ]:
radii = {
    "S1": 0.124775, "S2": 0.4991, "S3": 0.1595, "S4": 0.200525, "S5": 0.40105, 
    "S6": 0.12575, "S7": 0.103
}

inertias = {
    "Igen": 3.9, "I1": 0.029, "I2": 25.6, "I3": 1.43, "I4": 1.165, "I5": 4.99, 
    "I6": 1.43, "I7": 1.5, "mps": 40
}

friction = {
    "Bgen": 7, "Bdrivetrain": 40, "Bshaft": 40, "Bspring_pulley": 80, 
    "Bpneumatic_spring": 700, "Bpneumatic_spring_static1": 0, 
    "Bpspneumatic_spring_static2": 0
}

airspring = {
    "gamma": 1.4, "height": 1, "diameter": 3, "area": 0.0709676, 
    "press_init": 854e3, "vol_init": 1
}

gear_ratios = {
    "R21": radii['S2']/radii['S1'],
    "R45": radii['S4']/radii['S5'], 
    "R67": radii['S6']/radii['S7'],
    "spring": radii['S6']*(radii['S4']/radii['S5'])
}

inertia_PTO = (
    (inertias["Igen"]  + inertias["I1"])*gear_ratios['R21']**2 +
    (inertias['I2'] +inertias['I3'] + inertias['I4']) +
    gear_ratios["R45"]**2 * (
        inertias['I5'] + inertias['I6'] +
        inertias["I7"] * gear_ratios['R67']**2 +
        inertias['mps'] * radii['S6']**2   
    )
)

friction_PTO = (
    friction['Bgen']*gear_ratios['R21']**2 + 
    friction['Bdrivetrain'] +
    gear_ratios["R45"]**2 * (
        friction["Bshaft"]+
        friction["Bspring_pulley"]*gear_ratios['R67']**2 +
        friction["Bpneumatic_spring"]*radii['S6']**2
    )
)

##### Generator efficiency map
The generator used by the AquaHarmonics devices is a motor from a Nissan Leaf.
Nissan (and Oak Ridge National Lab) have published efficiency data on the motor.
Here, we have fit a polynomial to the discrete efficiency map provided by Nissan.
The advantage of this polynomial is that it is differentiable, and thus will allow the usage of automatic differentiation to obtain the optimal control trajectory solution.

In [ ]:
def efficiency(flow, effort):
    eff_max = 300
    flow_max = 10000*2*np.pi/60
    a = 1.148
    b = -0.4589
    c = -0.297
    d = 3.204
    e = -1.695
    f = 0.01
    o = -3.683

    efficiency = []
    for eff, flo in zip(effort/eff_max, flow/flow_max):
        efficiency.append(
            (1.2-f/(55*flo**2+0.377)**(3/2)) *
            (1.2-f/(230*eff**2+0.367)**(3/2))
            * (
                (a**3*flo**2*eff**2) + b**3*eff**2 + c**3.*flo**2 +
                e**5*eff**4*flo**4 + d
            )
            + o
        )

    return np.array(efficiency)

In addition to the efficiency data, we can also include limitations on the system.
Here, we will define:

 * Maximum rotational speed
 * Maximum torque
 * Maximum power

In [ ]:
rot_max = 10000*2*np.pi/60 # [rad/s]
torque_max = 300 # [Nm]
power_max = 80e3 # [W]

Finally, we can plot the efficiency surface to see how it looks.
Here, the independent variables are motor speed in [rad/s] and motor torque in [Nm].
Note that we limit the plot by power limit.

In [ ]:
x = np.arange(-1*rot_max, 1*rot_max, 10)
y = np.arange(-1*torque_max, 1.0*torque_max, 5)
X, Y = np.meshgrid(x, y)
Z = efficiency(X, Y).copy()
Z[np.abs(X*Y) > power_max] = np.NaN  # cut off area outside of power limit

fig = plt.figure(figsize=plt.figaspect(0.4))
ax = [fig.add_subplot(1, 2, 1, projection="3d"),
      fig.add_subplot(1, 2, 2)]

ax[0].plot_surface(X, Y, Z, cmap=cm.coolwarm,linewidth=0)
ax[0].set_xlabel('Shaft speed [rad/s]')
ax[0].set_ylabel('Torque [Nm]')
ax[0].set_zlabel('Efficiency')
ax[0].set_zlim([0, 1.1])

contour = ax[1].contourf(X, Y, Z)
plt.colorbar(contour, label="Efficiency")
ax[1].set_xlabel('Shaft speed [rad/s]')
ax[1].set_ylabel('Torque [Nm]')

plt.tight_layout()

##### PTO object
Using both the parametric kinematics (gear ratios) and the efficiency map, we can now create a PTO object that will be inserted in our WEC object.
Note that the friction and inertia effects will be included as additional forces.
By defining the PTO object in this way, we will be able to more clearly distinguish the power generating forces for other effects (e.g., friction).

In [ ]:
name = ["PTO_Heave",]
gear_ratio_generator = gear_ratios['R21']/radii['S3']
gear_ratio_generator = 249.5    # which one is correct???
kinematics = gear_ratio_generator*np.eye(ndof)
controller = None
nstate_opt = 2*nfreq + 1
pto_impedance = None
pto = wot.pto.PTO(
    ndof, kinematics, controller, pto_impedance, efficiency, name
)

#### Additional Forces
Now we can define other forces on the WEC:

 * **Buoyancy, gravitym and pretension** - While the buoyancy and gravity phenomena are often lumped together, we will keep them seperate here. This is useful because the AquaHarmonics devices has a pretension force, so the buoyancy and gravity forces are not balances at the neutral position with the pretension force from the tether/air spring.
* **Passive PTO forces** - Inertia forces due to finite rigid body inertia of gears and friction forces within the drive train are captured by this function.
* **PTO line force** - The total force imposed on the WEC from its tether is the sum of the PTO force due to action by the generator and the pretension force from the air spring.
 

In [ ]:
def F_buoyancy(wec, x_wec, x_opt, waves, nsubsteps = 1):
    """Only the zero-th order component (doesn't include linear stiffness"""
    return displacement * rho * g * np.ones([wec.ncomponents*nsubsteps, wec.ndof])

def F_gravity(wec, x_wec, x_opt, waves, nsubsteps = 1):
    return -1 * wec.inertia_matrix.item() * g * np.ones([wec.ncomponents*nsubsteps, wec.ndof])

def F_pretension_wec(wec, x_wec, x_opt, waves, nsubsteps = 1):
    """Pretension force as it acts on the WEC"""
    F_b = F_buoyancy(wec, x_wec, x_opt, waves, nsubsteps) 
    F_g = F_gravity(wec, x_wec, x_opt, waves, nsubsteps)
    return  -1*(F_b+F_g)

def F_pto_passive(wec, x_wec, x_opt, waves, nsubsteps = 1):
    pos = wec.vec_to_dofmat(x_wec)
    vel = np.dot(wec.derivative_mat,pos)
    acc = np.dot(wec.derivative_mat, vel)
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    spring = -(gear_ratios['spring']*airspring['gamma']*airspring['area']*
              airspring['press_init']/airspring['vol_init']) * pos
    F_spring = np.dot(time_matrix,spring)
    fric = -(friction_PTO  + 
                friction['Bpneumatic_spring_static1']*
                gear_ratios['spring']) * vel
    F_fric = np.dot(time_matrix,fric)
    inertia = inertia_PTO * acc
    F_inertia = np.dot(time_matrix,inertia)
    return F_spring + F_fric + F_inertia

def F_pto_line(wec, x_wec, x_opt, waves, nsubsteps = 1):
    f_pto = pto.force_on_wec(wec, x_wec, x_opt, waves, nsubsteps)
    f_pre = F_pretension_wec(wec, x_wec, x_opt, waves, nsubsteps)
    return f_pto + f_pre

f_add = {'PTO': F_pto_line,
         'PTO_passive': F_pto_passive,
         'buoyancy': F_buoyancy,
         'gravity': F_gravity}

#### Constraints
A number a constraints need to be imposed to reflect finite limitation of the drive train hardware.
Each of these will be implemented as inequality constraints that will be passed to the numerical optimization solver.

 * **Peak torque** - The absolute motor torque cannot exceed this value.
 * **Maximum rotational speed** - The absolute motor speed cannot exceed this value.
 * **Maximum power** - The maximum mechanical power (i.e., product of torque and velocity) cannot exceed this value.
 * **Minumum line tension** - In addition to these limitations on the hardware, we will also constrain the solution to prevent the tether tension from going below a certain threshold. We absolutely cannot allow the line tension to be less than zero, or else it will go slack. In practice, it is prudent to set some nonzero limit for the tether tension.

In [ ]:
# Generator constraints
torque_peak_max = 280    #[Nm]   
rot_speed_max = 10000*2*np.pi/60    #[rad/s]
# mooring line constraint
min_line_tension = -1000

nsubsteps = 10

def const_peak_torque_pto(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
    """Instantaneous torque must not exceed max torque 
        Tmax - |T| >=0 """
    torque = pto.force(wec, x_wec, x_opt, waves, nsubsteps)
    return torque_peak_max - np.abs(torque.flatten())

def const_speed_pto(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
    rot_vel = pto.velocity(wec, x_wec, x_opt, waves, nsubsteps)
    return rot_speed_max - np.abs(rot_vel.flatten())

def const_power_pto(wec, x_wec, x_opt, waves): # Format for scipy.optimize.minimize
    power_mech = (pto.velocity(wec, x_wec, x_opt, waves, nsubsteps) *
                    pto.force(wec, x_wec, x_opt, waves, nsubsteps))

    return power_max - np.abs(power_mech.flatten())

def constrain_min_tension(wec, x_wec, x_opt, waves):
    total_tension = -1*F_pto_line(wec, x_wec, x_opt, waves, nsubsteps)
    return total_tension.flatten() + min_line_tension


constraints = [{'type': 'ineq','fun': constrain_min_tension,},
               {'type': 'ineq','fun': const_peak_torque_pto,},
               {'type': 'ineq','fun': const_speed_pto,},
               {'type': 'ineq','fun': const_power_pto,}]


#### WEC object
Finally, we can use the different components we've developed to construct a `WEC` object:

 * **BEM data** - defines the hydrodynamics of the hull
 * **Inertia and hydrostatics** - rigid-body inertia and hydrostatic stiffness
 * **Constraints** - limitations on the hardware (max power, max torque, etc.) and the constraint to prevent the tether from going slack
 * **Additional forces** - this captures all of the forces on the WEC that are not due to the interaction between the hull and water (PTO, friction, etc.)

In [ ]:
wec = wot.WEC.from_bem(
    bem_data,
    inertia_matrix=mass,
    hydrostatic_stiffness=stiffness,
    constraints=constraints,
    friction=None,
    f_add=f_add,
)

### Waves
A regular wave will allow us to get a good understanding of the optimal control trajectory.
Note that we'll want to choose a wave frequency that is within the frequency array we used to calculate the hydrodynamic data.

In [ ]:
amplitude = 0.25 
wavefreq =  0.24 #6*f1 
phase = 30
wavedir = 0
waves = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude, phase, wavedir)

### Objective function
To drive the solution of the optimal control trajectory, we will optimize the average electrical power from the PTO.

In [ ]:
obj_fun = pto.average_power

### Solve
Finally, we solve for the optimal control trajectory.
To ensure the numerical optimization problem is well-scaled, we set [specific scaling factors](https://snl-waterpower.github.io/WecOptTool/theory.html#scaling) for the WEC position (`scale_x_wec`), the PTO force (`scale_x_opt`), and objective function (`scale_obj`).
We will also set the [maximum iteration and objective function tolerance for the optimization solver](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-lbfgsb.html).
This step typically takes 10-15s.

In [ ]:
scale_x_wec = 1e1
scale_x_opt = 50e-2
scale_obj = 1e-3

options = {'maxiter': 200, 'ftol':1e-8} 

results = wec.solve(
    waves, 
    obj_fun, 
    nstate_opt,
    x_wec_0=np.ones(wec.nstate_wec)*1e-3,
    x_opt_0=np.ones(nstate_opt)*1e-3,
    optim_options=options, 
    scale_x_wec=scale_x_wec,
    scale_x_opt=scale_x_opt,
    scale_obj=scale_obj,
    )

print(f'Optimal average power: {results.fun} W')

In [ ]:
results

### Post-process and plotting
Next, we post-process the results to allow us visualize them in a series of plots.
Note that because the `WEC` and `PTO` objects are distinct (the WEC object really only knows what the force from the PTO is, not how it is obtained), we create two separate sets results objects (in each case, we get results in the frequency domain and time domain).

In [ ]:
wec_fdom, wec_tdom = wec.post_process(results, waves, nsubsteps=nsubsteps)
pto_fdom, pto_tdom = pto.post_process(wec, results, waves, nsubsteps=nsubsteps)
wec_tdom

#### Time histories
We will now generate a series of time history plots:

 * **Excitation force and velocity** - Looking at the velocity of the WEC along with the excitation force is useful as we expect the two of these to be in phase when maximizing mechanical power and in the absence of constraints.
 * **PTO and total mooring tether force** - Since we constrained the PTO force in order to maintain a minimum mooring tether tension, it is useful to visualize these values to confirm that the constraint was properly enforced.
 * **Generator torque** - Similarly, we constrained the maximum torque from the generator
 * **Power** - We can look at both electrical and mechanical power along with the constraint on mechanical power.

In [ ]:
fig, ax = plt.subplots(nrows=4,
                       sharex=True,
                       figsize=(12, 12))

# Excitation and velocity
ax1 = ax[0].twinx()
(wec_tdom.force.sel(type=['Froude_Krylov', 'diffraction']).sum('type')/1e3).plot(ax=ax[0], color='k')
wec_tdom.vel.plot(ax=ax1, color='orange')
ax1.set_ylabel('Velocity [m/s]', color='orange')
ax1.tick_params(axis='y', labelcolor='orange')
ax1.set_title('')
ax1.autoscale(enable=True, axis='x', tight=False)
ax[0].set_ylabel('Excitation force [kN]')

# Forces
(wec_tdom.force.sel(type='PTO')/1e3).plot(ax=ax[1], 
                                          label='PTO force in WEC frame')
x_wec, x_opt = wot.decompose_state(results.x, ndof=ndof, nfreq=nfreq)
ax[1].plot(wec.time_nsubsteps(nsubsteps),
           F_pto_line(wec, x_wec, x_opt, waves, nsubsteps)/1e3,
           linestyle='dashed', label='Mooring line tension')
ax[1].plot(wec.time, min_line_tension*np.ones(wec.time.shape)/1e3,
           linestyle='dotted', color='black',
           label=f'Minimum line tension ({min_line_tension/1e3:.0f}kN)')
ax[1].axhline(y=0, xmin=0, xmax=1, color='0.75', linewidth=0.5)
ax[1].set_ylabel('Force [kN]')
ax[1].legend(loc=1)

# Torque
pto_tdom.force.plot(ax=ax[2], linestyle='solid',
                    label='PTO torque in PTO frame')
ax[2].plot(pto_tdom.time, 1*torque_peak_max*np.ones(pto_tdom.time.shape),
           color='black', linestyle='dotted',
           label=f'Peak torque limit ($\pm${torque_peak_max}Nm)')
ax[2].plot(pto_tdom.time, -1*torque_peak_max *
           np.ones(pto_tdom.time.shape), color='black', linestyle='dotted')
ax[2].set_ylabel('Torque [Nm] ')
ax[2].legend(loc='upper right',)

# Power
(pto_tdom['mech_power']/1e3).plot(ax=ax[3], label='Mech. power')
(pto_tdom['power']/1e3).plot(ax=ax[3], linestyle='dashed', label='Elec. power')
ax[3].axhline(-1*power_max/1e3,
              linestyle='dotted', color='black', 
              label=f'Max mech. power ($\pm${power_max/1e3:.0f}kW)')
ax[3].axhline(1*power_max/1e3,
              linestyle='dotted', color='black')
ax[3].grid(color='0.75', linestyle='-',
           linewidth=0.5, axis='x')
ax[3].legend(loc='upper right')
ax[3].set_title('')
ax[3].set_ylabel('Power [kW]')

for axi in ax:
    axi.set_title('')
    axi.grid()
    axi.label_outer()
    axi.autoscale(axis='x', tight=True)


#### Generator torque vs. shaft speed
An important factor in this study was the efficiency map that we defined for the generator.
We can visualize how the optimal control trajectory looks along with that efficiency map.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,6))
ax.plot(pto_tdom.vel, pto_tdom.force, color='k')
contour = ax.contourf(X, Y, Z)
plt.colorbar(contour, label="Efficiency")
ax.grid(which='major', linestyle='--')
ax.axvline(0, color='k', lw=1)
ax.axhline(0, color='k', lw=1)
ax.set_ylim([-torque_max, torque_max])
ax.set_ylabel('Torque [Nm]')
ax.set_xlabel('Shaft speed [rad/s]')

## 2. Control co-design of line pre-tension with non-linear power conversion
Design study: optimal mass vs pretension. Control co-design: for each design considered find it's optimal control strategy.

First, create a function that encapsulates everything from Part 1. The input is the mass of the device, and the output is the average electric power produced. 

In [ ]:
def design_obj_fun(x):
    global n
    n += 1

    # Unpack geometry variables
    mass_var = x[0]
    displacement = displaced_mass_cpy/rho
    
    #BEM
    bem_data = bem_file
    
    #PTO
    name = ["PTO_Heave",]
    kinematics = gear_ratio_generator*np.eye(ndof)
    controller = None
    nstate_opt = 2*nfreq+1
    
    # loss = loss_interp
    pto_impedance = None
    pto = wot.pto.PTO(ndof, kinematics, controller, pto_impedance, efficiency, name)
    obj_fun = pto.average_power
    
    #forces
    def F_gravity(wec, x_wec, x_opt, waves, nsubsteps = 1):
        return -1 * wec.inertia_matrix.item() * g * np.ones([wec.ncomponents*nsubsteps, wec.ndof])

    def F_buoyancy(wec, x_wec, x_opt, waves, nsubsteps = 1):
        """Only the zero-th order component (doesn't include linear stiffness"""
        return displacement * rho * g * np.ones([wec.ncomponents*nsubsteps, wec.ndof])

    def F_pretension_wec(wec, x_wec, x_opt, waves, nsubsteps = 1):
        """Pretension force as it acts on the WEC"""
        F_b = F_buoyancy(wec, x_wec, x_opt, waves, nsubsteps) 
        F_g = F_gravity(wec, x_wec, x_opt, waves, nsubsteps)
        return  -1*(F_b+F_g)

    def F_pto_line(wec, x_wec, x_opt, waves, nsubsteps = 1):
        f_pto = pto.force_on_wec(wec, x_wec, x_opt, waves, nsubsteps)
        f_pre = F_pretension_wec(wec, x_wec, x_opt, waves, nsubsteps)
        return f_pto + f_pre

    def F_pto_passive(wec, x_wec, x_opt, waves, nsubsteps = 1):
        pos = wec.vec_to_dofmat(x_wec)
        vel = np.dot(wec.derivative_mat,pos)
        acc = np.dot(wec.derivative_mat, vel)
        time_matrix = wec.time_mat_nsubsteps(nsubsteps)
        spring = -(gear_ratios['spring']*airspring['gamma']*airspring['area']*
                airspring['press_init']/airspring['vol_init']) * pos
        F_spring = np.dot(time_matrix,spring)
        fric = -(friction_PTO  + 
                    friction['Bpneumatic_spring_static1']*
                    gear_ratios['spring']) * vel
        F_fric = np.dot(time_matrix,fric)
        inertia = inertia_PTO * acc
        F_inertia = np.dot(time_matrix,inertia)
        return F_spring + F_fric + F_inertia

    #constraints
    torque_peak_max = 280    #[Nm]   
    torque_continues_max = 120 #[Nm]
    rot_speed_max = 10000*2*np.pi/60    #[rad/s]
    power_max = 80000   #[W]
    min_line_tension = -1000
    # nsubsteps = 5

    def const_peak_torque_pto(wec, x_wec, x_opt, waves): 
        """Instantaneous torque must not exceed max torque 
            Tmax - |T| >=0 """
        torque = pto.force(wec, x_wec, x_opt, waves, nsubsteps)
        return torque_peak_max - np.abs(torque.flatten())

    ineq_cons_peak_torque = {'type': 'ineq', 'fun': const_peak_torque_pto}

    # def const_torque_pto(wec, x_wec, x_opt, waves): 
    #     """RMS torque must not exceed max continous torque 
    #         Tmax_conti - Trms >=0 """
    #     torque_rms = np.sqrt(np.mean(pto.force(wec, x_wec, x_opt, waves, nsubsteps)**2))
    #     return torque_continues_max - np.abs(torque_rms.flatten())
    # 
    # ineq_cons_torque = {'type': 'ineq', 'fun': const_torque_pto}

    def const_speed_pto(wec, x_wec, x_opt, waves):
        rot_vel = pto.velocity(wec, x_wec, x_opt, waves, nsubsteps)
        return rot_speed_max - np.abs(rot_vel.flatten())

    ineq_cons_rot_speed = {'type': 'ineq', 'fun': const_speed_pto}

    def const_power_pto(wec, x_wec, x_opt, waves):
        power_mech = (pto.velocity(wec, x_wec, x_opt, waves, nsubsteps) *
                        pto.force(wec, x_wec, x_opt, waves, nsubsteps))
        return power_max - np.abs(power_mech.flatten())

    ineq_cons_power = {'type': 'ineq', 'fun': const_power_pto}

    def constrain_min_tension(wec, x_wec, x_opt, waves):
        total_tension = -1*F_pto_line(wec, x_wec, x_opt, waves, nsubsteps)
        return total_tension.flatten() + min_line_tension

    ineq_cons_tension = {'type': 'ineq', 'fun': constrain_min_tension}

    constraints = [
        ineq_cons_tension, 
        # ineq_cons_torque, 
        ineq_cons_peak_torque, 
        ineq_cons_rot_speed, 
        ineq_cons_power]
     
    f_add_var = {'PTO': F_pto_line,
         'PTO_passive': F_pto_passive,
         'buoyancy': F_buoyancy,
         'gravity': F_gravity}

    # update WEC 
    wec_mass = wot.WEC.from_bem(
                bem_data,
                inertia_matrix= mass_var,
                hydrostatic_stiffness=stiffness,
                constraints=constraints,
                friction=None,
                f_add=f_add_var,
            )

    # Solve
    print(f'Run {n} off {N}: Mass: {mass_var}kg')
    # scale_x_wec = 1e1
    # scale_x_opt = 1e-2*1 
    # scale_obj = 1e-3 
    res = wec_mass.solve(
        waves, 
        obj_fun, 
        nstate_opt, 
        optim_options=options, 
        scale_x_wec=scale_x_wec,
        scale_x_opt=scale_x_opt,
        scale_obj=scale_obj)
    opt_average_power = res.fun
    print(f'Mass: {mass_var}kg, Optimal average power: {opt_average_power} W')
    return res.fun

decide on the range of mass/pretension to look at. 
The max mass is the one that results in the min pre-tension being met at still water. 
We should not run it at that high of a mass, but we use it to non-dimensionlize mass (a mass ratio of 1 is mass equal to this max).

In [ ]:
min_ten = -1000
max_mass = (min_ten/g + displaced_mass_cpy).item()

global n, N
n = 0
N = 7
min_mass_ratio = 0.3
max_mass_ratio = 1.0
mass_vals = np.linspace(
    min_mass_ratio*max_mass, 
    max_mass_ratio*max_mass, 
    N, 
    endpoint=False
)
ranges = (slice(
    mass_vals[0], 
    mass_vals[-1]+np.diff(mass_vals)[0], 
    np.diff(mass_vals)[0]),
)

### Solve

In [ ]:
res = brute(func=design_obj_fun, ranges=ranges, full_output=True,  finish=None)

In [ ]:
fig, ax = plt.subplots()
ax.plot(res[2]/max_mass, res[3], 'o-')
ax.set_xlabel("$m/m_{max}$")
ax.set_ylabel("$\overline{P}_e$ [W]")
ax.set_ylim(top=0.0)

Summary: It has little effect.